<a href="https://colab.research.google.com/github/johncoogan53/XAI_HW2/blob/main/XAI_HW_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIPI 590 - XAI | Assignment 02

Description:

John Coogan